In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import theano
import matplotlib
from matplotlib import pyplot
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
import logging 
log = logging.getLogger()
log.setLevel('DEBUG')

Using gpu device 0: GeForce GTX TITAN Black (CNMeM is disabled, CuDNN 3007)


# Experiment Tutorial

In this tutorial, we will run an experiment to train a model for online BCI decoding.

We have these five steps for the experiment:

1. Setup the dataset
2. Setup model
3. Setup data splitting and iteration
4. Setup other experiment parameters (loss, optimizater, monitoring etc.)
5. Run the experiment :)

You have to adjust file paths :)

## Setup of dataset

In [3]:
from braindecode.datasets.combined import CombinedCntSets
from braindecode.mywyrm.processing import resample_cnt, bandpass_cnt, exponential_standardize_cnt

In [4]:
nico_C_sensors = ['FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6',
          'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1', 'CPz',
          'CP2', 'CP4', 'CP6']
markers_10_class = {'1- Right Hand': [1], '2 - Left Hand': [2], '3 - Rest': [3],
           '4 - Feet': [4], '5 - Face': [5], '6 - Navigation': [6], '7 - Music': [7],
            '8 - Rotation': [8], '9 - Subtraction': [9], '10 - Words': [10]}

In [5]:
# Set args define one sequence per set: 
# filename, loader class (or 'bbci'), (i_first_trial, i_last_trial), (trial_start_ms, trial_end_ms),
# end marker dictionary
# trials up to before i_last_trial are contained, i_last_trial is not contained 
# end_marker_dictionary is only necessary if trial length is not fixed trial length
# it is a dictionary mapping start marker codes to end marker codes for all classes
# In case you have end marker definitions, trial end defines the offset to the end marker
# e.g. -500ms means 500ms before the trial end marker

set_args = (('data/robot-hall/NiRiNBD2S001R02_ds10_1-8.BBCI.mat',
                           'bbci', (0,600), (1500,4000), None),
          ('data/robot-hall/NiRiNBD2S001R02_ds10_1-8.BBCI.mat',
           'bbci', (600,800), (0,4000), None))
load_sensor_names = nico_C_sensors # In case you only want to load specific sensors, even before preprocessing 
sensor_names = nico_C_sensors # sensor names actually used for decoding, selected after preprocessing,
# useful in case you need a sensor during preprocessing which you do not want to use for decoding

# for preprocessing of continuous signal,
# define list of functions and arguments, functions will be applied one after the other:
cnt_preprocessors=((resample_cnt, dict(newfs=100)), # just to be faster in this tutorial :)
                   (bandpass_cnt, dict(low_cut_hz=4, high_cut_hz=None, filt_order=4)),
                   (exponential_standardize_cnt, dict())
                   )

marker_def=markers_10_class
dataset = CombinedCntSets(set_args=set_args,
                          load_sensor_names=load_sensor_names,
                          sensor_names=sensor_names,
                          cnt_preprocessors=cnt_preprocessors,
                          marker_def=marker_def
                         )

preprocessor = None # a later optional generic preprocessor for any time of dataset, not needed here

### Setup of model

In [6]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet

In [7]:
input_time_length = 750 # determines crops predicted simultaneously
model = ShallowFBCSPNet(in_chans=len(sensor_names), input_time_length=input_time_length,n_classes=10)
final_layer = model.get_layers()[-1]

In [8]:
### We can also print the layers
from braindecode.veganlasagne.layer_util import print_layers
print_layers(final_layer)

 0 InputLayer                                                         [None, 21, 750, 1]
 1 DimshuffleLayer                                                    (None, 1, 750, 21)
 2 Conv2DLayer              25x1                                      (None, 40, 726, 21)
 3 Conv2DAllColsLayer       1x21                                      (None, 40, 726, 1)
 4 BatchNormLayer                                     square         
 5 Pool2DLayer              75x1                      average_exc_pad (None, 40, 652, 1)
 6 NonlinearityLayer                                  safe_log       
 7 StrideReshapeLayer           ::15 ::1                              (None, 40, 44, 1)
 8 DropoutLayer                                                      
 9 Conv2DLayer              30x1                                      (None, 10, 15, 1)
10 FinalReshapeLayer                                                  (None, 10)
11 NonlinearityLayer                                  softmax        



### Setup data splitting and iteration

In [9]:
# How to split the set into train valid and test
from braindecode.datahandling.splitters import SeveralSetsSplitter
from braindecode.veganlasagne.layers import get_n_sample_preds
# several sets splitter splits of the last set as a test set, 
# valid set fraction says what fraction of the training set is used for validation/early stopping
splitter = SeveralSetsSplitter(valid_set_fraction=0.2)

from braindecode.datahandling.batch_iteration import CntWindowTrialIterator
# iterator defines how to extract one training batch from the data
# CntWindowTrialIterator cut out crops of the trial as described in paper :)
# get n sample preds computes how many samples the Model above will predict
# in a single forward pass, i.e. input time length - receptive field + 1
iterator = CntWindowTrialIterator(batch_size=60, input_time_length=input_time_length,
                                 n_sample_preds=get_n_sample_preds(final_layer))



### Setup experiment/optimization parameters

In [10]:
from lasagne.objectives import categorical_crossentropy
from lasagne.updates import adam
from braindecode.veganlasagne.stopping import MaxEpochs, NoDecrease, Or
from braindecode.veganlasagne.monitors import RuntimeMonitor, LossMonitor, MisclassMonitor, CntTrialMisclassMonitor
from braindecode.veganlasagne.update_modifiers import MaxNormConstraint
# Standard multi-class loss function
loss_expression = categorical_crossentropy
# how to compute updates given loss and params or grads and params
updates_expression = adam

# Modify updates which are computed by the updates expression
# Typically used to constrain weights for each layer to some norm
# (alternative to weight decay, which would be part of the loss function)
updates_modifier = MaxNormConstraint(default_norm=2.)

# Logging of expressions, both for output and to be used by early stopping
# note that the normal misclass monitor directly compared outputs and targets
# which in this case are predictions and labels for each sample, so we give it a different 
# channel name
# Loss, Misclass and CntTrialMisclass monitor automatically prepend the dataset name
# to the output, e.g., train_sample_misclass, valid_sample_misclass, ...
monitors = [LossMonitor(),MisclassMonitor(chan_name='sample_misclass'),
           CntTrialMisclassMonitor(input_time_length=input_time_length),
           RuntimeMonitor()]

# when to do the first stop, I usually do a combination of early stopping
# and a hard stop criterion (here with untypically small numbers):
stop_criterion = Or(stop_criteria=
                    (NoDecrease(chan_name='valid_misclass', num_epochs=15, min_decrease=0.),
                     MaxEpochs(25)))

# Which channel to use to select the best model after the early stop
remember_best_chan = 'valid_misclass'
run_after_early_stop = True # Could also just stop then without using validation part of training set
batch_modifier = None # object modifying a training batch, e.g., for online data augmentation


### Setup and run the experiment

In [11]:
from braindecode.experiments.experiment import Experiment

In [12]:
exp = Experiment(final_layer, dataset=dataset, splitter=splitter, preprocessor=preprocessor,
          iterator=iterator, loss_expression=loss_expression,
          updates_expression=updates_expression, updates_modifier=updates_modifier,
          monitors=monitors, stop_criterion=stop_criterion, remember_best_chan=remember_best_chan,
          run_after_early_stop=run_after_early_stop, batch_modifier=batch_modifier)

In [13]:
# yes it loads both twice, once for train and once for set... but still fast enough IMO :D
exp.setup() # also resets lasagnes random generator.

INFO:braindecode.datasets.combined:Loading set 1 of 2
INFO:braindecode.datasets.cnt_signal_matrix:Load continuous signal...
INFO:braindecode.datasets.cnt_signal_matrix:Preprocess continuous signal...
INFO:braindecode.mywyrm.processing:Using highpass filter since high cut hz is None
INFO:braindecode.datasets.cnt_signal_matrix:Loaded dataset with shape: (454370, 21, 1, 1)
INFO:braindecode.datasets.combined:Loading set 2 of 2
INFO:braindecode.datasets.cnt_signal_matrix:Load continuous signal...
INFO:braindecode.datasets.cnt_signal_matrix:Preprocess continuous signal...
INFO:braindecode.mywyrm.processing:Using highpass filter since high cut hz is None
INFO:braindecode.datasets.cnt_signal_matrix:Loaded dataset with shape: (151296, 21, 1, 1)
INFO:braindecode.experiments.experiment:Layers...
INFO:braindecode.experiments.experiment:
 0 InputLayer                                                         [None, 21, 750, 1]
 1 DimshuffleLayer                                                    (Non

In [14]:
exp.run()

INFO:braindecode.experiments.experiment:Run until first stop...
INFO:braindecode.experiments.experiment:Split/Preprocess datasets...
INFO:braindecode.experiments.experiment:...Done
INFO:braindecode.experiments.experiment:Epoch 0
INFO:braindecode.experiments.experiment:train_loss                9.40613
INFO:braindecode.experiments.experiment:valid_loss                9.55397
INFO:braindecode.experiments.experiment:test_loss                 9.40684
INFO:braindecode.experiments.experiment:train_sample_misclass     0.90149
INFO:braindecode.experiments.experiment:valid_sample_misclass     0.89831
INFO:braindecode.experiments.experiment:test_sample_misclass      0.89950
INFO:braindecode.experiments.experiment:train_misclass            0.90149
INFO:braindecode.experiments.experiment:valid_misclass            0.89831
INFO:braindecode.experiments.experiment:test_misclass             0.89950
INFO:braindecode.experiments.experiment:runtime                   0.00000
INFO:braindecode.experiments.ex

## With Config-File

Now we can run some similar experiments with a yaml-config file.


#### The config file
This is the config file :) It uses yaml plus dollar-variables to define templates and parameters. it is a bit hacky :) Possibly it should be replaced by https://github.com/IDSIA/sacred one day :)

In [ ]:
# %load configs/experiments/tutorial/shallow_net_online.yaml
{
    templates: {
        resample_bandpass_standardize: 
        [[
                !!python/name:braindecode.mywyrm.processing.resample_cnt , 
                {'newfs': $resample_fs},
            ],
            [
                !!python/name:braindecode.mywyrm.processing.bandpass_cnt , 
                {'low_cut_hz': $low_cut_off_hz,
                'high_cut_hz': $high_cut_hz,
                filt_order: $filt_order}
        ],
            [
                !!python/name:braindecode.mywyrm.processing.exponential_standardize_cnt , 
                { }
        ],
            ],
        combined_cnt_set: !obj:braindecode.datasets.combined.CombinedCntSets {
            set_args: !!python/object/apply:zip [
                $filenames, $set_types, 
                $trial_ranges, $all_segment_ivals, $end_marker_defs],
            load_sensor_names: $load_sensor_names,
            sensor_names: $sensor_names,
            marker_def: $marker_def,
            cnt_preprocessors: $cnt_preprocessors,
        },
        # not used 
        # end_markers_movement_class: {'01 - Right Hand': [5], 
        #'02 - Left Hand': [6], '03 - Rest': [7],
        #   '04 - Feet': [8},
        markers_10_class: {'1- Right Hand': [1], '2 - Left Hand': [2], '3 - Rest': [3],
           '4 - Feet': [4], '5 - Face': [5], '6 - Navigation': [6], '7 - Music': [7],
            '8 - Rotation': [8], '9 - Subtraction': [9], '10 - Words': [10] },
        markers_movement_class:  {'01 - Right Hand': [1], '02 - Left Hand': [2], '03 - Rest': [3],
           '04 - Feet': [4]},
         nico_C_sensors: ['FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6',
          'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1', 'CPz',
          'CP2', 'CP4', 'CP6'],
          # in_sensors will be replaced automatically 
          shallow_net: !obj:braindecode.models.shallow_fbcsp.ShallowFBCSPNet {
            in_chans: in_sensors,
            input_time_length: $input_time_length,
            n_classes: $n_classes,
          },
          several_sets_splitter: !obj:braindecode.datahandling.splitters.SeveralSetsSplitter {
            use_test_as_valid: $use_test_as_valid,
            valid_set_fraction: $valid_set_fraction,
        },
        cnt_iterator: !obj:braindecode.datahandling.batch_iteration.CntWindowTrialIterator {
            batch_size: $batch_size,
            input_time_length: $input_time_length,
            n_sample_preds: $n_sample_preds,
        },
        sample_trial_monitors: [
                !obj:braindecode.veganlasagne.monitors.LossMonitor {},
                !obj:braindecode.veganlasagne.monitors.MisclassMonitor {
                    chan_name: 'sample_misclass'},
                !obj:braindecode.veganlasagne.monitors.CntTrialMisclassMonitor { 
                    input_time_length: $input_time_length},
                !obj:braindecode.veganlasagne.monitors.RuntimeMonitor {},
        ],
        categorical_crossentropy: !!python/name:lasagne.objectives.categorical_crossentropy ,
        adam: !!python/name:lasagne.updates.adam ,
        max_norm_modifier: !obj:braindecode.veganlasagne.update_modifiers.MaxNormConstraint {
            default_norm: $default_norm,
        },
       
    },
    variants: [[{ 
        # where results will be saved
        save_path: ['data/models/online/tutorial/'],
        
        filenames: [[  
                'data/robot-hall/NiRiNBD2S001R02_ds10_1-8.BBCI.mat',
                'data/robot-hall/NiRiNBD2S001R02_ds10_1-8.BBCI.mat',
        ]],
        set_types: [[
            'bbci',
            'bbci',
        ]],
        all_segment_ivals: [[
            [1500, 4500],
            [0, 4000],
            ],
        ],
        # no end markers
        end_marker_defs: ["[ 
                null,
                null,
        ]"],
        load_sensor_names: [$nico_C_sensors],
        sensor_names: [$nico_C_sensors],
        cnt_preprocessors: [$resample_bandpass_standardize],
        resample_fs: [250],
        
        # try both 0 and 4
        low_cut_off_hz: [0,4],
        high_cut_hz: ['null'],
        filt_order: [4],
        
        dataset: [$combined_cnt_set],
        dataset_splitter: [$several_sets_splitter],
        
        preprocessor: ['null'],
        
        layers: [$shallow_net],
        input_time_length: [750],
        
        valid_set_fraction: [0.2],
        use_test_as_valid: [False],
        
        batch_size: [60],
        n_sample_preds: [200], # will be replaced automatically
        
        iterator: [$cnt_iterator],
        monitors: [$sample_trial_monitors],

        loss_expression: [$categorical_crossentropy],
        updates_expression: [$adam],
        updates_modifier: [$max_norm_modifier],
        default_norm: [2.],

        early_stop_chan: ['valid_misclass'],
        max_increasing_epochs: [15],
        max_epochs: [25],
        run_after_early_stop: [true],
        batch_modifier: ['null'],
        variants: [[
        # with 10 classes
        {
            # null indicates till end
            trial_ranges: [
            "[[0,600], [600,null]]"
            ],
            marker_def: [$markers_10_class],
            n_classes: [10],
        }, 
        # with only movement classes,
        # adjust trial ranges...
        {
            trial_ranges: [
            "[[0,300], [300,null]]"
            ],
            marker_def: [$markers_movement_class],
            n_classes: [4],
        }]],
        # not used but needs to be given
        num_cv_folds: [-1],
        
    }]],
}

To run this config file run this in the terminal, in the braindecode folder:

```./scripts/train_experiments.py configs/experiments/tutorial/shallow_net_online.yaml```

You can also run specific experiments

```./scripts/train_experiments.py configs/experiments/tutorial/shallow_net_online.yaml --start 1 --stop 1
./scripts/train_experiments.py configs/experiments/tutorial/shallow_net_online.yaml --start 2 --stop 4```

etc.



In the config directory you can see a lot of config fiels with more advanced usages like extending other config files etc.

### Analyzing experiments run this way

In [20]:
from braindecode.analysis.pandas_util import load_data_frame, remove_columns_with_same_value

In [21]:
remove_columns_with_same_value(load_data_frame('data/models/tutorial/online/'))

,marker_def,trial_ranges,low_cut_off_hz,n_classes,time,test,test_sample,train,train_sample
1,markers_10_class,"[[0,600], [600,null]]",0,10,00:05:02,36.683417,29.750596,99.332220,93.052168
4,markers_10_class,"[[0,600], [600,null]]",4,10,00:04:46,36.180905,28.121251,99.332220,92.623843
5,markers_movement_class,"[[0,300], [300,null]]",0,4,00:02:11,57.894737,48.847926,98.662207,93.032458
6,markers_movement_class,"[[0,300], [300,null]]",4,4,00:02:17,52.631579,41.993694,100.000000,97.162970


## Stuff just for me (Robin) :)

In [1]:
%%capture
import scikits.samplerate
import os
import site
site.addsitedir('/home/schirrmr/.local/lib/python2.7/site-packages/')
site.addsitedir('/usr/lib/pymodules/python2.7/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
%cd /home/schirrmr/braindecode/code/braindecode/
assert 'THEANO_FLAGS' in os.environ
# switch to cpu
#os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'